# Predicting Life Satisfaction with NHS Data: Which Lifestyle factors are most important for LS?

Website: https://www.cdc.gov/nchs/nhis/documentation/2023-nhis.html

Dataset: C:\Users\sacar\OneDrive\Documents\Projects\Predicting MH with NHS Data\adult23.csv

Description: 



In [119]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [120]:
#Import data
df = pd.read_csv(r'C:\Users\sacar\OneDrive\Documents\Projects\Predicting MH with NHS Data\adult23.csv')

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29522 entries, 0 to 29521
Columns: 647 entries, URBRRL to POVRATTC_A
dtypes: float64(442), int64(204), object(1)
memory usage: 145.7+ MB


In [122]:
df.head(2)

,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,LANGSPECR_A,LANGSOC_A,LANGDOC_A,LANGMED_A,LANGHM_A,PPSU,...,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A
0,3,4,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,7371.139,H029691,1.01
1,4,8,0,0,NaN,NaN,NaN,NaN,NaN,2,...,NaN,NaN,1,1,1,10,1,3146.794,H028812,2.49


In [123]:
df.describe()

,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,LANGSPECR_A,LANGSOC_A,LANGDOC_A,LANGMED_A,LANGHM_A,PPSU,...,PHSTAT_A,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,POVRATTC_A
count,29522.000000,29522.000000,29522.000000,29522.000000,4049.000000,3973.000000,4049.000000,4049.000000,22104.000000,29522.000000,...,29522.000000,537.000000,553.000000,29522.000000,29522.0,29522.000000,29522.0,29522.0,29522.000000,29522.000000
mean,2.317119,9.666757,0.041664,0.373721,1.453445,1.480997,1.301309,1.392937,2.063563,31.375246,...,2.460369,1.286778,1.039783,1.205542,1.0,6.472089,10.0,1.0,8747.291918,4.106340
std,1.061522,4.048065,0.199823,0.712244,0.801351,0.790779,0.631127,0.734160,1.287790,29.253976,...,1.074983,0.668921,0.338166,1.087867,0.0,3.444791,0.0,0.0,5690.650182,2.961649
min,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,10.0,1.0,1792.441000,0.000000
25%,1.000000,7.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000,8.000000,...,2.000000,1.000000,1.000000,1.000000,1.0,3.000000,10.0,1.0,4643.531750,1.800000
50%,2.000000,10.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000,24.000000,...,2.000000,1.000000,1.000000,1.000000,1.0,7.000000,10.0,1.0,7374.546000,3.310000
75%,3.000000,14.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,48.000000,...,3.000000,1.000000,1.000000,1.000000,1.0,9.000000,10.0,1.0,10994.809500,5.650000
max,4.000000,14.000000,1.000000,2.000000,9.000000,9.000000,9.000000,9.000000,9.000000,153.000000,...,9.000000,4.000000,8.000000,8.000000,1.0,12.000000,10.0,1.0,39925.600000,11.000000


In [124]:
df_copy = df.copy()

# Pre-Processing

# Missing Values

In [125]:
# Calculate missing value percentages
missing_percent = df_copy.isnull().sum() / len(df_copy) * 100

# Print summary
print(missing_percent.describe())

count    647.000000
mean      52.059685
std       42.190768
min        0.000000
25%        0.000000
50%       62.421245
75%       93.630174
max      100.000000
dtype: float64


In [126]:
# Check missing values for all columns and display the count and percentage
missing_values = df_copy.isnull().sum()
missing_percentage = (missing_values / len(df_copy)) * 100

# Count columns with missing values
missing_columns_count = (df_copy.isnull().sum() > 0).sum()
print(f"Number of columns with missing values: {missing_columns_count}")


# Filter only columns with missing values
missing_data = missing_values[missing_values > 0]

# Print missing values and percentages
print("Columns with Missing Values:")
for col in missing_data.index:
    print(f"{col}: {missing_data[col]} missing ({missing_percentage[col]:.2f}%)")


Number of columns with missing values: 440
Columns with Missing Values:
LANGSPECR_A: 25473 missing (86.28%)
LANGSOC_A: 25549 missing (86.54%)
LANGDOC_A: 25473 missing (86.28%)
LANGMED_A: 25473 missing (86.28%)
LANGHM_A: 7418 missing (25.13%)
SCHDYMSSTC_A: 27881 missing (94.44%)
AFNOW: 8099 missing (27.43%)
REPWRKDYTC_A: 28193 missing (95.50%)
YRSINUS_A: 24820 missing (84.07%)
PRTNREDUCP_A: 27652 missing (93.67%)
SPOUSEDUCP_A: 17305 missing (58.62%)
SASPPRACE_A: 15434 missing (52.28%)
SASPPHISP_A: 15434 missing (52.28%)
PRTNRAGETC_A: 27651 missing (93.66%)
SPOUSAGETC_A: 17305 missing (58.62%)
PRTNRWKFT_A: 28118 missing (95.24%)
PRTNRWRK_A: 27660 missing (93.69%)
SPOUSWKFT_A: 21989 missing (74.48%)
SPOUSWRK_A: 17356 missing (58.79%)
SPOUSESEX_A: 17306 missing (58.62%)
PRTNRSEX_A: 27651 missing (93.66%)
INJWRKDYTC_A: 27562 missing (93.36%)
NUMINJTC_A: 27562 missing (93.36%)
SHINGYEARP_A: 22275 missing (75.45%)
HHRESPSA_FLG: 8637 missing (29.26%)
EPINUMSEZP_A: 28954 missing (98.08%)
EMPDYS

In [127]:
# Recalculate missing percentages based on df_copy
missing_percent = (df_copy.isnull().sum() / len(df_copy)) * 100  

# Select only columns where missing percentage is less than 50%
df_copy = df_copy.loc[:, missing_percent < 50]

print(f"Remaining columns after dropping: {df_copy.shape[1]}")

Remaining columns after dropping: 294


In [128]:
# Keep rows that have at least 70% valid data

#df_copy = df_copy.dropna(thresh=int(df.shape[1] * 0.5))  


# Drop single categorical column for now

In [129]:
df_copy = df_copy.select_dtypes(exclude=['object'])

In [130]:
# Replace NaNs with median values

df_copy.fillna(df_copy.median(), inplace=True)  

In [131]:
print("Remaining missing values:", df_copy.isnull().sum().sum())  # Should print 0


Remaining missing values: 0


In [ ]:
print(f"Total missing values before imputation: {df.isnull().sum().sum()}")
print(f"Total missing values after imputation: {df_copy.isnull().sum().sum()}")  # Should be 0

#Note that this isn't true many were actually just dropped


Total missing values before imputation: 9943782
Total missing values after imputation: 0


# Train/Test Split

In [133]:
# Separate target variable (Life Satisfaction)
y = df_copy["LSATIS4_A"]  # Target variable
X = df_copy.drop(columns=["LSATIS4_A"])  # Features (everything except target)

# Confirm separation
print(f"Feature Set (X): {X.shape}")
print(f"Target (y): {y.shape}")


Feature Set (X): (29522, 292)
Target (y): (29522,)


In [95]:
#Train/Test Split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.3, 
                                                  random_state = 0, 
                                                  stratify = y, 
                                                  shuffle = True)

print(f"X_train shape: {X_train.shape}, X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}, y_val shape: {y_val.shape}")

X_train shape: (20665, 292), X_val shape: (8857, 292)
y_train shape: (20665,), y_val shape: (8857,)


In [ ]:
#Drop columns with missing values and non-numeric columns
#X_train = X_train.select_dtypes(include='number')
#X_train = X_train.dropna(axis=1)

# Ensure same columns in validation set
#X_val = X_val[X_train.columns]  # Ensure same columns in validation set

# Feature Selection

In [38]:
# Compute correlation matrix
corr_matrix = df_copy.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find columns with correlation > 0.9
high_corr_features = [column for column in upper.columns if any(upper[column] > 0.9)]

# Drop highly correlated features
df_copy = df_copy.drop(columns=high_corr_features)

print(f"Dropped {len(high_corr_features)} highly correlated features.")


Dropped 0 highly correlated features.


In [ ]:
# Initialize a model (Random Forest for feature selection)
rf = RandomForestRegressor(n_estimators=100, random_state=0)

# Perform RFE to select top 20 features
rfe = RFE(estimator=rf, n_features_to_select=20)
rfe.fit(X_train, y_train)

# Keep only the selected features
selected_features_rfe = X_train.columns[rfe.support_]
X_train_rfe = X_train[selected_features_rfe]
X_val_rfe = X_val[selected_features_rfe]

print(f"Selected {len(selected_features_rfe)} features using RFE.")

# Modelling

# Linear Regression

In [11]:
model = LinearRegression()

In [12]:
model.fit(X_train, y_train)

LinearRegression()

In [13]:
predictions = model.predict(X_val)
predictions

array([2.43560251, 1.25477953, 1.47590333, ..., 1.5791727 , 1.02513756,
       1.38532131])

In [14]:
# Calculate predictions for both training and validation sets
train_predictions = model.predict(X_train)
val_predictions = model.predict(X_val)

# Calculate evaluation metrics for training set
train_mae = mean_absolute_error(y_train, train_predictions)
train_mse = mean_squared_error(y_train, train_predictions)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, train_predictions)

# Calculate evaluation metrics for validation set
val_mae = mean_absolute_error(y_val, val_predictions)
val_mse = mean_squared_error(y_val, val_predictions)
val_rmse = np.sqrt(val_mse)
val_r2 = r2_score(y_val, val_predictions)

# Print results for training and validation sets
print("🔹 Training Set Metrics:")
print(f"Mean Absolute Error (MAE): {train_mae:.4f}")
print(f"Mean Squared Error (MSE): {train_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {train_rmse:.4f}")
print(f"R² Score: {train_r2:.4f}\n")

print("🔹 Validation Set Metrics:")
print(f"Mean Absolute Error (MAE): {val_mae:.4f}")
print(f"Mean Squared Error (MSE): {val_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {val_rmse:.4f}")
print(f"R² Score: {val_r2:.4f}")


🔹 Training Set Metrics:
Mean Absolute Error (MAE): 0.4392
Mean Squared Error (MSE): 0.3447
Root Mean Squared Error (RMSE): 0.5871
R² Score: 0.2689

🔹 Validation Set Metrics:
Mean Absolute Error (MAE): 0.4495
Mean Squared Error (MSE): 0.3638
Root Mean Squared Error (RMSE): 0.6031
R² Score: 0.2278


In [15]:
# Get feature importance from model coefficients (for Linear Regression)
feature_importance = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": model.coef_
})

# Sort by absolute coefficient value (most important first)
feature_importance["Abs_Coefficient"] = feature_importance["Coefficient"].abs()
feature_importance = feature_importance.sort_values(by="Abs_Coefficient", ascending=False)

# Display the top 20 most important features
print("Top 20 Most Important Features:")
print(feature_importance.head(20))


Top 20 Most Important Features:
          Feature  Coefficient  Abs_Coefficient
33         CHIP_A    -0.310883         0.310883
150    HIKIND07_A     0.275841         0.275841
31          IHS_A    -0.241593         0.241593
197      PHSTAT_A     0.146455         0.146455
152    HIKIND05_A     0.106269         0.106269
191       STREV_A    -0.088309         0.088309
24     PCNT18UPTC    -0.083287         0.083287
184     DEMENEV_A     0.079963         0.079963
107       PHQ42_A     0.076982         0.076982
113     DEPFREQ_A    -0.071777         0.071777
192        MIEV_A     0.067566         0.067566
134    MEDDL12M_A    -0.064623         0.064623
178      PSOREV_A    -0.059176         0.059176
148    HIKIND09_A     0.059033         0.059033
177       CFSEV_A     0.057343         0.057343
55     TRANSPOR_A    -0.056124         0.056124
42     PCNTADLT_A     0.055730         0.055730
40    MLTFAMFLG_A    -0.053319         0.053319
68    NATUSBORN_A     0.049873         0.049873
16   EMP

# Random Forest

In [16]:
# Initialize and train the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [17]:
# Make predictions
rf_train_preds = rf_model.predict(X_train)
rf_val_preds = rf_model.predict(X_val)

In [18]:
# Evaluate
def evaluate(y_true, y_pred, dataset="Validation"):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"🔹 {dataset} Set Metrics:")
    print(f"  - MAE: {mae:.4f}")
    print(f"  - RMSE: {rmse:.4f}")
    print(f"  - R²: {r2:.4f}\n")

evaluate(y_train, rf_train_preds, dataset="Training")
evaluate(y_val, rf_val_preds, dataset="Validation")

🔹 Training Set Metrics:
  - MAE: 0.1649
  - RMSE: 0.2288
  - R²: 0.8889

🔹 Validation Set Metrics:
  - MAE: 0.4479
  - RMSE: 0.6075
  - R²: 0.2165

